#### 1. Import library

In [3]:
import json 
import pandas as pd 
import numpy as np 
from typing import Dict, List, Tuple 
import sqlite3
import json

#### 2. Load the data onto DataFrame

In [4]:
# load the json file of one month onto dataframe 
df_data = pd.read_json('../data/airlines-1558527599826.json', lines=True)

#### 3. Store the data on .db file 

In [7]:
# create connection and cursor 
con = sqlite3.connect('../database/sqlite-tools-win32-x86-3380300/database.db')
cur = con.cursor()

#### 4. KLM table 

In [68]:
df_datas = df_data.dropna(subset = ["user"])
#Extract only tweets made by KLM (including posting +replying to other):
test = df_datas.copy()

ind = []
for index, row in df_datas.iterrows():
    if row['user']['id_str'] == '56377143':
        ind.append(index)
klm_tweet = df_datas.loc[ind]

In [69]:
#Extract only tweets mention KLM (@KLM etc...):
test2 = df_datas.copy()
ind2 = []
for index, row in df_datas.iterrows():
    if "@KLM" in row['text']:
        ind2.append(index)
klm_tweet_men = df_datas.loc[ind2]

In [70]:
#Concat 2 tables:
klm_table = pd.concat([klm_tweet, klm_tweet_men])
klm_table.drop_duplicates(subset=['id'], inplace=True)

In [71]:
#Export the table as CSV file to import to the database
klm_table.to_csv('../database/CSV file/klm_table.csv')

#### 5. Bristish Airways table

In [62]:
#Extract only tweets made by BA (including posting +replying to other):
test = df_datas.copy()

ind = []
for index, row in df_datas.iterrows():
    if row['user']['id_str'] == '18332190':
        ind.append(index)
british_tweet = df_datas.loc[ind]

In [72]:
#Extract only tweets mention BA (@British_Airways etc...):
test2 = df_datas.copy()
ind2 = []
for index, row in df_datas.iterrows():
    if "@British_Airways" in row['text']:
        ind2.append(index)
british_tweet_men = df_datas.loc[ind2]

In [76]:
#Concat 2 tables:
british_table = pd.concat([british_tweet, british_tweet_men])
british_table.drop_duplicates(subset=['id'], inplace=True)

In [77]:
#Export the table as CSV file to import to the database
british_table.to_csv('../database/CSV file/british_table.csv')

#### 6. Users table 

In [99]:
df_datas.user

0        {'id': 393374091, 'id_str': '393374091', 'name...
1        {'id': 3420691215, 'id_str': '3420691215', 'na...
2        {'id': 394376606, 'id_str': '394376606', 'name...
3        {'id': 36488556, 'id_str': '36488556', 'name':...
4        {'id': 14193348, 'id_str': '14193348', 'name':...
                               ...                        
14384    {'id': 1499872951, 'id_str': '1499872951', 'na...
14385    {'id': 2182054841, 'id_str': '2182054841', 'na...
14386    {'id': 1000339516203421696, 'id_str': '1000339...
14387    {'id': 128174143, 'id_str': '128174143', 'name...
14388    {'id': 2285714020, 'id_str': '2285714020', 'na...
Name: user, Length: 14387, dtype: object

In [101]:
#Get the user table
#This is a temporary method as it's really bad and have bad running time @Phat
user_table = pd.concat([pd.DataFrame(df_datas.user[i], index=[i]) for i in df_datas.user.index])

In [103]:
user_table.to_csv('../database/CSV file/user_table.csv')